In [2]:
#Import libraries required
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
#Load up guide RNA sequences and target DNA sequences as pandas dataframe
df_guide_RNA = pd.read_csv(r"C:\Users\dasak\OneDrive\Documents\GitHub\cas12-collateral-cleavage-prediction\full_guide_RNA_sequences.csv")
df_target_DNA = pd.read_csv(r"C:\Users\dasak\OneDrive\Documents\GitHub\cas12-collateral-cleavage-prediction\target_DNA_sequences.csv")
df_target_DNA_reverse_complements = pd.read_csv(r"C:\Users\dasak\OneDrive\Documents\GitHub\cas12-collateral-cleavage-prediction\target_DNA_sequences_reverse_complements.csv")


In [4]:
# Load k_value_sigmoid from experiment on 24_05_2022 
from numpy import genfromtxt
k_values = genfromtxt('k_values_sigmoid.csv', delimiter=',')

In [5]:
def one_hot_encode_RNA(gRNA_sequence):
    mapping = dict(zip("aucg", range(4)))    
    one_hot_encoded_sequence = [mapping[i] for i in gRNA_sequence]
    return np.eye(4)[one_hot_encoded_sequence]

In [6]:
def one_hot_encode_DNA(DNA_sequence):
    mapping = dict(zip("atcg", range(4)))    
    one_hot_encoded_sequence = [mapping[i] for i in DNA_sequence]
    return np.eye(4)[one_hot_encoded_sequence]

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import mean_squared_error
from scipy import stats

def train_dt(input_features, outputs, input_feature_names):
    '''
    This function takes in input features and outputs as kvalue for an experiment. 
    It then trains a decision tree on these features.
    
    '''
    input_train, input_test, output_train, output_test = train_test_split(
        input_features, outputs, test_size=0.2, random_state=42)
    model = tree.DecisionTreeRegressor(random_state = 42)
    model = model.fit(input_train, output_train)
    model_importance = pd.DataFrame({'Feature_names':input_feature_names,'Importance': model.feature_importances_})
    output_predictions = model.predict(input_test)
    mse = mean_squared_error(output_test, output_predictions)
    pearson_rank_results = stats.pearsonr(output_test, output_predictions)
    return(model_importance,mse, pearson_rank_results)
    
    
    


In [ ]:
#One hot encode guide RNA sequences
guide_RNA_sequences = df_guide_RNA['Sequence']
guide_RNA_one_hot_encoded_sequences = []
for i in guide_RNA_sequences:
    one_hot_encoded_sequence = one_hot_encode_RNA(i)
    guide_RNA_one_hot_encoded_sequences.append(one_hot_encoded_sequence.ravel())

#generate feature names
feature_names_gRNA = []
for i in range(41):
    feature_names_gRNA.append('guide RNA A' + str(i+1))
    feature_names_gRNA.append('guide RNA U' + str(i+1))
    feature_names_gRNA.append('guide RNA C' + str(i+1))
    feature_names_gRNA.append('guide RNA G' + str(i+1))

dt_gRNA_model_importance = []
dt_gRNA_mse =[]
dt_gRNA_model_importance, dt_gRNA_mse, dt_gRNA_pearson_results = train_dt(guide_RNA_one_hot_encoded_sequences,k_values,feature_names_gRNA)
print(dt_gRNA_model_importance.sort_values(by = 'Importance', ascending = False))
print(dt_gRNA_mse)
print(dt_gRNA_pearson_results)

     Feature_names  Importance
109  guide RNA U28    0.424395
87   guide RNA G22    0.347347
102  guide RNA C26    0.044627
148  guide RNA A38    0.017392
139  guide RNA G35    0.014928
..             ...         ...
55   guide RNA G14    0.000000
56   guide RNA A15    0.000000
57   guide RNA U15    0.000000
58   guide RNA C15    0.000000
163  guide RNA G41    0.000000

[164 rows x 2 columns]
0.026148952604759843
(0.30697356885981697, 0.12716325804638254)


In [8]:
#One hot encode only the 20 gRNA bases that interact with the target DNA
complementary_guide_RNA_sequences = []
for i in guide_RNA_sequences:
    complementary_guide_RNA_sequence = i[21:41]
    complementary_guide_RNA_sequences.append(complementary_guide_RNA_sequence)

#One hot encode them
complementary_guide_RNA_one_hot_encoded_sequences = []
for i in complementary_guide_RNA_sequences:
    one_hot_encoded_sequence = one_hot_encode_RNA(i).ravel()
    complementary_guide_RNA_one_hot_encoded_sequences.append(one_hot_encoded_sequence)

#generate feature names
feature_names_complementary_gRNA = []
for i in range(20):
    feature_names_complementary_gRNA.append('guide RNA A' + str(i+1))
    feature_names_complementary_gRNA.append('guide RNA U' + str(i+1))
    feature_names_complementary_gRNA.append('guide RNA C' + str(i+1))
    feature_names_complementary_gRNA.append('guide RNA G' + str(i+1))

dt_complementary_gRNA_model_importance = []
dt_complementary_gRNA_mse =[]
dt_complementary_gRNA_model_importance, dt_complementary_gRNA_mse, dt_complementary_gRNA_pearson_results = train_dt(complementary_guide_RNA_one_hot_encoded_sequences,k_values, feature_names_complementary_gRNA)
print(dt_complementary_gRNA_model_importance.sort_values(by = 'Importance', ascending = False))
print(dt_complementary_gRNA_mse)
print(dt_complementary_gRNA_pearson_results)
    
    

    Feature_names  Importance
37  guide RNA U10    0.422723
3    guide RNA G1    0.347708
18   guide RNA C5    0.044627
64  guide RNA A17    0.017488
55  guide RNA G14    0.014928
..            ...         ...
50  guide RNA C13    0.000000
51  guide RNA G13    0.000000
54  guide RNA C14    0.000000
10   guide RNA C3    0.000000
79  guide RNA G20    0.000000

[80 rows x 2 columns]
0.0245816333043721
(0.3201836447333988, 0.11079377287155823)


In [9]:
#Try 8 bit vectors where the first 4 bits represent the guide RNA and the second 4 bits represent the target DNA. Use only the 20bp matching region to begin with.
#Take the last twenty bases of the guide RNA sequences
complementary_guide_RNA_sequences = []
for i in guide_RNA_sequences:
    complementary_guide_RNA_sequence = i[21:41]
    complementary_guide_RNA_sequences.append(complementary_guide_RNA_sequence)

#One hot encode them
complementary_guide_RNA_one_hot_encoded_sequences = []
for i in complementary_guide_RNA_sequences:
    one_hot_encoded_sequence = one_hot_encode_RNA(i)
    complementary_guide_RNA_one_hot_encoded_sequences.append(one_hot_encoded_sequence)


#Take the middle 20 of the target complements and reorder back to fron so they become complementary to guide RNA sequence
target_DNA_sequences = df_target_DNA['Sequence']
target_DNA_complements = []
complementary_target_DNA_sequences = []
for i in target_DNA_sequences:
    complementary_target_DNA_sequence = i[39:19:-1] #This generates the complement to the guide
    complementary_target_DNA_sequences.append(complementary_target_DNA_sequence)
# print(complementary_target_DNA_sequences[0])    

complementary_target_DNA_one_hot_encoded_sequences = []
for i in complementary_target_DNA_sequences:
    one_hot_encoded_sequence = one_hot_encode_DNA(i)
    complementary_target_DNA_one_hot_encoded_sequences.append(one_hot_encoded_sequence)
# print(complementary_target_DNA_one_hot_encoded_sequences[0])  

#Concatenate the one hot encodes sequences
concat_complementary_guide_RNAs_target_DNAs = []
# test with 1st element 
for i in range(len(complementary_guide_RNA_one_hot_encoded_sequences)):
    # print(i)
    concat_complementary_guide_RNA_target_DNA = np.hstack((complementary_guide_RNA_one_hot_encoded_sequences[i], complementary_target_DNA_one_hot_encoded_sequences[i])).ravel()
    concat_complementary_guide_RNAs_target_DNAs.append(concat_complementary_guide_RNA_target_DNA)

#Set up feature names 
feature_names_concat_guide_RNA_target_DNA = []
for i in range(20):
    feature_names_concat_guide_RNA_target_DNA.append('guide RNA A' + str(i+1))
    feature_names_concat_guide_RNA_target_DNA.append('guide RNA U' + str(i+1))
    feature_names_concat_guide_RNA_target_DNA.append('guide RNA C' + str(i+1))
    feature_names_concat_guide_RNA_target_DNA.append('guide RNA G' + str(i+1))
    feature_names_concat_guide_RNA_target_DNA.append('target DNA A' + str(i+1))
    feature_names_concat_guide_RNA_target_DNA.append('target DNA T' + str(i+1))
    feature_names_concat_guide_RNA_target_DNA.append('target DNA C' + str(i+1))
    feature_names_concat_guide_RNA_target_DNA.append('target DNA G' + str(i+1))
    
dt_concat_importance=[]
dt_concat_mse = []
dt_concat_pearson_results = []
dt_concat_importance, dt_concat_mse, dt_concat_pearson_results = train_dt(concat_complementary_guide_RNAs_target_DNAs, k_values, feature_names_concat_guide_RNA_target_DNA)
print(dt_concat_importance.sort_values(by = 'Importance', ascending = False))
print(dt_concat_mse)
print(dt_concat_pearson_results)


      Feature_names  Importance
76   target DNA A10    0.421721
3      guide RNA G1    0.347266
39    target DNA G5    0.044627
128   guide RNA A17    0.017392
107   guide RNA G14    0.014928
..              ...         ...
61    target DNA T8    0.000000
59     guide RNA G8    0.000000
55    target DNA G7    0.000000
54    target DNA C7    0.000000
159  target DNA G20    0.000000

[160 rows x 2 columns]
0.03286427437692059
(0.3401300223702354, 0.08910275603369928)


In [10]:
#Add context dependency of 4 bp either side of the target DNA. This represents the PAM site
#Make a array of zeros to add to either side of the gRNA one hot encoded sequence. This signifies the area around
#the target DNA where the gRNA should (hopefully!)not bind.

zeros = np.zeros((4,4))
pam_included_guide_RNA_one_hot_encoded_sequence = []
pam_included_guide_RNA_one_hot_encoded_sequences = []
for i in complementary_guide_RNA_sequences:
    pam_included_guide_RNA_one_hot_encoded_sequence = np.vstack((zeros,one_hot_encode_RNA(i)))
    pam_included_guide_RNA_one_hot_encoded_sequences.append(pam_included_guide_RNA_one_hot_encoded_sequence)

#one hot encode full 60 bp target DNA sequence so that they line up as complement to the gRNA sequence 
#with context included

pam_included_target_DNA_sequence = []
pam_included_target_DNA_sequences = []
for i in target_DNA_sequences:
    pam_included_target_DNA_sequence = i[43:19:-1] #This generates the complement to the guide
    pam_included_target_DNA_sequences.append(pam_included_target_DNA_sequence)

pam_included_target_DNA_one_hot_encoded_sequences = []
for i in pam_included_target_DNA_sequences:
    one_hot_encoded_sequence = one_hot_encode_DNA(i)
    pam_included_target_DNA_one_hot_encoded_sequences.append(one_hot_encoded_sequence)

    
# Concatenate the one hot encodes sequences
concat_pam_included_guide_RNA_target_DNA = []
concat_pam_included_guide_RNAs_target_DNAs = []
for i in range(len(pam_included_target_DNA_one_hot_encoded_sequences)):
    concat_pam_included_guide_RNA_target_DNA = np.hstack((pam_included_guide_RNA_one_hot_encoded_sequences[i], pam_included_target_DNA_one_hot_encoded_sequences[i])).ravel()
    concat_pam_included_guide_RNAs_target_DNAs.append(concat_pam_included_guide_RNA_target_DNA)

# #Add feature names. Note guide RNA 1-4 denote non pairing as no binding should happen there
feature_names_pam_included_guide_RNA_target_DNA = []
for i in range(24):
    feature_names_pam_included_guide_RNA_target_DNA.append('guide RNA A' + str(i+1))
    feature_names_pam_included_guide_RNA_target_DNA.append('guide RNA U' + str(i+1))
    feature_names_pam_included_guide_RNA_target_DNA.append('guide RNA C' + str(i+1))
    feature_names_pam_included_guide_RNA_target_DNA.append('guide RNA G' + str(i+1))
    feature_names_pam_included_guide_RNA_target_DNA.append('target DNA A' + str(i+1))
    feature_names_pam_included_guide_RNA_target_DNA.append('target DNA T' + str(i+1))
    feature_names_pam_included_guide_RNA_target_DNA.append('target DNA C' + str(i+1))
    feature_names_pam_included_guide_RNA_target_DNA.append('target DNA G' + str(i+1))



# #Train the dt 
dt_pam_included_importance=[]
dt_pam_included_mse = []
dt_pam_included_pearson_results = []
dt_pam_included_importance, dt_pam_included_mse, dt_pam_included_pearson_results = train_dt(concat_pam_included_guide_RNAs_target_DNAs, k_values, feature_names_pam_included_guide_RNA_target_DNA)
print(dt_pam_included_importance.sort_values(by = 'Importance', ascending = False))
print(dt_pam_included_mse)
print(dt_pam_included_pearson_results)

      Feature_names  Importance
66     guide RNA C9    0.762854
28    target DNA A4    0.054117
169   guide RNA U22    0.018290
70    target DNA C9    0.017698
58     guide RNA C8    0.015042
..              ...         ...
72    guide RNA A10    0.000000
73    guide RNA U10    0.000000
74    guide RNA C10    0.000000
77   target DNA T10    0.000000
191  target DNA G24    0.000000

[192 rows x 2 columns]
0.5885813305715738
(0.046399917925027845, 0.8219177954795238)


In [11]:
#Add context dependency of 20 bp either side of the target DNA
#Make a array of zeros to add to either side of the gRNA one hot encoded sequence. This signifies the area around
#the target DNA where the gRNA should (hopefully!) not bind.
zeros = np.zeros((20,4))
context_included_guide_RNA_one_hot_encoded_sequence = []
context_included_guide_RNA_one_hot_encoded_sequences = []
for i in complementary_guide_RNA_sequences:
    context_included_guide_RNA_one_hot_encoded_sequence = np.vstack((zeros,one_hot_encode_RNA(i),zeros))
    context_included_guide_RNA_one_hot_encoded_sequences.append(context_included_guide_RNA_one_hot_encoded_sequence)

#one hot encode full 60 bp target DNA sequence so that they line up as complement to the gRNA sequence with context
# included

len(target_DNA_sequences[0])

full_target_DNA_sequence = []
full_target_DNA_sequences = []
for i in target_DNA_sequences:
    full_target_DNA_sequence = i[::-1] #This generates the complement to the guide
    full_target_DNA_sequences.append(full_target_DNA_sequence)

full_target_DNA_one_hot_encoded_sequences = []
for i in full_target_DNA_sequences:
    one_hot_encoded_sequence = one_hot_encode_DNA(i)
    full_target_DNA_one_hot_encoded_sequences.append(one_hot_encoded_sequence)

    
# Concatenate the one hot encodes sequences
concat_context_included_guide_RNA_target_DNA = []
concat_context_included_guide_RNAs_target_DNAs = []
for i in range(len(full_target_DNA_one_hot_encoded_sequences)):
    concat_context_included_guide_RNA_target_DNA = np.hstack((context_included_guide_RNA_one_hot_encoded_sequences[i], full_target_DNA_one_hot_encoded_sequences[i])).ravel()
    concat_context_included_guide_RNAs_target_DNAs.append(concat_context_included_guide_RNA_target_DNA)

#Add feature names. Note guide RNA 1-20 and 41-60 denote non pairing as no binding should happen there
feature_names_context_included_guide_RNA_target_DNA = []
for i in range(60):
    feature_names_context_included_guide_RNA_target_DNA.append('guide RNA A' + str(i+1))
    feature_names_context_included_guide_RNA_target_DNA.append('guide RNA U' + str(i+1))
    feature_names_context_included_guide_RNA_target_DNA.append('guide RNA C' + str(i+1))
    feature_names_context_included_guide_RNA_target_DNA.append('guide RNA G' + str(i+1))
    feature_names_context_included_guide_RNA_target_DNA.append('target DNA A' + str(i+1))
    feature_names_context_included_guide_RNA_target_DNA.append('target DNA T' + str(i+1))
    feature_names_context_included_guide_RNA_target_DNA.append('target DNA C' + str(i+1))
    feature_names_context_included_guide_RNA_target_DNA.append('target DNA G' + str(i+1))



#Train the dt 
dt_context_included_importance=[]
dt_context_included_mse = []
dt_context_included_pearson_results = []
dt_context_included_importance, dt_context_included_mse, dt_context_included_pearson_results = train_dt(concat_context_included_guide_RNAs_target_DNAs, k_values, feature_names_context_included_guide_RNA_target_DNA)
print(dt_context_included_importance.sort_values(by = 'Importance', ascending = False))
print(dt_context_included_mse)
print(dt_context_included_pearson_results)

      Feature_names  Importance
194   guide RNA C25    0.762849
156  target DNA A20    0.054117
444  target DNA A56    0.022017
111  target DNA G14    0.021852
45    target DNA T6    0.020581
..              ...         ...
161   guide RNA U21    0.000000
160   guide RNA A21    0.000000
159  target DNA G20    0.000000
158  target DNA C20    0.000000
479  target DNA G60    0.000000

[480 rows x 2 columns]
0.6207887504580195
(-0.018229029783011427, 0.9295699993364074)


In [14]:
#Generate a bar plot for mse and pearson for each cases
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1)
fig['layout']['yaxis']['title']='Mean squared error'
fig['layout']['yaxis2']['title']='Pearson correlation coefficient'
categories = ['Full sequence guide RNA',
         'Target DNA-complementary guide RNA sequence', 
         'Target DNA-complementary guide RNA and guide RNA-complementary target DNA',
         'Target DNA-complementary guide RNA and guide RNA-complementary target DNA (PAM included]',
         'Target DNA-complementary guide RNA and guide RNA-complementary target DNA (Full included)']

fig.add_trace(go.Bar(x=categories, y=[dt_gRNA_mse, dt_complementary_gRNA_mse,dt_concat_mse, dt_pam_included_mse, dt_context_included_mse]), row=1, col=1)
fig.add_trace(go.Bar(x=categories, y=[dt_gRNA_pearson_results[0], dt_complementary_gRNA_pearson_results[0],dt_concat_pearson_results[0], dt_pam_included_pearson_results[0], dt_context_included_pearson_results[0]]), row=2, col=1)
fig.update_xaxes(tickangle=-0, tickfont=dict(size=16))
fig.update_layout(height = 1000, width = 5000, plot_bgcolor='white',showlegend = False)

fig.show()

In [21]:
# Look at non sequence based features to begin with
# GC content of guide RNA sequence
def calculate_GC_content(sequence):
    count = 0
    for i in range(len(sequence)):
        if sequence[i] == 'g' or sequence[i] == 'c':
            count += 1
    return count/len(sequence)







In [32]:
guide_RNA_sequences = df_guide_RNA['Sequence']
guide_RNAs_gc_content = []

for i in guide_RNA_sequences:
    guide_RNA_gc_content = calculate_GC_content(i)
    guide_RNAs_gc_content.append(guide_RNA_gc_content)

feature_name_gc = 'GC content'

guide_RNAs_gc_content[0]

0.43902439024390244

In [52]:
#Add gc content to each of the feature names
#One hot encode guide RNA sequences
guide_RNA_sequences = df_guide_RNA['Sequence']
guide_RNA_one_hot_encoded_sequences = []
for i in guide_RNA_sequences:
    one_hot_encoded_sequence = one_hot_encode_RNA(i)
    guide_RNA_one_hot_encoded_sequences.append(one_hot_encoded_sequence.ravel())


#Add gc contentent as a feature at the end of each one hot encoded gc 
guide_RNA_one_hot_encoded_sequences_gc_added = []
for i in range(len(guide_RNA_one_hot_encoded_sequences)):
    guide_RNA_one_hot_encoded_sequence_gc_added = np.append(guide_RNA_one_hot_encoded_sequences[i], guide_RNAs_gc_content[i])
    guide_RNA_one_hot_encoded_sequences_gc_added.append(guide_RNA_one_hot_encoded_sequence_gc_added)

#generate feature names
feature_names_gRNA = []
for i in range(41):
    feature_names_gRNA.append('guide RNA A' + str(i+1))
    feature_names_gRNA.append('guide RNA U' + str(i+1))
    feature_names_gRNA.append('guide RNA C' + str(i+1))
    feature_names_gRNA.append('guide RNA G' + str(i+1))

feature_names_gRNA.append(feature_name_gc)

dt_gRNA_gc_added_model_importance = []
dt_gRNA_gc_added_mse =[]
dt_gRNA_gc_added_model_importance, dt_gRNA_gc_added_mse, dt_gRNA_gc_added_pearson_results = train_dt(guide_RNA_one_hot_encoded_sequences_gc_added,k_values,feature_names_gRNA)
print(dt_gRNA_gc_added_model_importance.sort_values(by = 'Importance', ascending = False))
print(dt_gRNA_gc_added_mse)
print(dt_gRNA_gc_added_pearson_results)

     Feature_names  Importance
143  guide RNA G36    0.421721
87   guide RNA G22    0.347266
102  guide RNA C26    0.044627
140  guide RNA A36    0.018364
164     GC content    0.017086
..             ...         ...
56   guide RNA A15    0.000000
57   guide RNA U15    0.000000
58   guide RNA C15    0.000000
59   guide RNA G15    0.000000
82   guide RNA C21    0.000000

[165 rows x 2 columns]
0.038294185979162655
(0.22393227056508302, 0.2714550530663572)
